# Generate Travel Time Matrix

In [1]:
import geopandas as gpd
import pandas as pd
import os
import datetime as dt
import time
from copy import deepcopy
import numpy as np
import osmnx as ox
import networkx as nx
import scipy
import sys
import pickle
import math

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

os.chdir(os.path.join(os.getcwd(), ".."))
print(os.getcwd())

/Volumes/external_drive/Code/paratransit-mdp


In [2]:
# Global VARS

DATA_DIR = os.path.join(os.getcwd(), "data")
OUTPUT_DIR = os.path.join(os.getcwd(), "data", "travel_time_matrix")

In [45]:
def osmnx_routing_graph(addr='Chattanooga, Tennessee', 
                        buffer_dist=500, 
                        strongly_connected=True, 
                        integer_edge_weights=True):
    """
    Uses OSMNX to get OSM routing graph for a city. Returns the routing graph (networkx)
    as well as the nodes and edges as pandas DataFrames. buffer_dist will expand the
    OSM network by buffer_dist (meters) in all directions. If strongly_connected then return
    the largest strongly connected component, if false return largest weakly connected component.
    If integer_edge_weights, then edge weights (travel time) is rounded up to nearest integer.
    
    :param addr: str
    :param buffer_dist: int
    :param strongly_connected: bool
    :param integer_edge_weights: bool
    :return: networkx.MultiDiGraph, pandas.DataFrame, pandas.DataFrame
    """
    # get the OSM graph
    G = ox.graph_from_place('Chattanooga, Tennessee',
                            network_type='drive',
                            simplify=True,
                            truncate_by_edge=True,
                            retain_all=False,
                            buffer_dist=buffer_dist)
    G = ox.utils_graph.get_largest_component(G, strongly=strongly_connected)

    # add edge speeds
    G = ox.speed.add_edge_speeds(G, fallback=40.2, precision=6)

    # add edge travel time
    G = ox.speed.add_edge_travel_times(G, precision=6)
    if integer_edge_weights:
        for n1, n2, k in G.edges(keys=True):
            G[n1][n2][k]['travel_time'] = math.ceil(G[n1][n2][k]['travel_time'])


    nodes, edges = ox.utils_graph.graph_to_gdfs(G)

    # format nodes
    nodes['osmid'] = nodes.index
    nodes.index = range(len(nodes))
    nodes['node_id'] = nodes.index
    nodes['lon'] = nodes['x']
    nodes['lat'] = nodes['y']
    nodes = nodes[['node_id', 'osmid', 'lat', 'lon']]
    nodes['node_id'] = nodes['node_id'].astype(int)
    nodes['osmid'] = nodes['osmid'].astype(int)
    nodes['lat'] = nodes['lat'].astype(float)
    nodes['lon'] = nodes['lon'].astype(float)

    # format edges
    edges = edges.reset_index()
    edges['source_osmid'] = edges['u']
    edges['target_osmid'] = edges['v']
    edges['source_node'] = edges['source_osmid'].apply(lambda x: nodes.loc[nodes['osmid']==x, 'node_id'].values[0])
    edges['target_node'] = edges['target_osmid'].apply(lambda x: nodes.loc[nodes['osmid']==x, 'node_id'].values[0])
    edges = edges.sort_values(by=['travel_time'])
    edges = edges.drop_duplicates(subset=['source_node', 'target_node'])
    edges = edges[['source_osmid', 'target_osmid', 'source_node', 'target_node', 'travel_time']]
    edges['source_osmid'] = edges['source_osmid'].astype(int)
    edges['target_osmid'] = edges['target_osmid'].astype(int)
    edges['source_node'] = edges['source_node'].astype(int)
    edges['target_node'] = edges['target_node'].astype(int)
    edges['travel_time'] = edges['travel_time'].astype(int)
    # format edge types
    print(f"Number of nodes: {len(nodes)}, number of edges: {len(edges)}")
    return G, nodes, edges


def osmnx_travel_time_matrix_from_graph(G, nodes, matrix_as_int=True):
    """
    Extracts a 2-dimensional travel time matrix from an osmnx routing graph.
    If matrix_as_int is True then returns a numpy array of type int32, else float32.
    :param G: networkx
    :param nodes: pandas.DataFrame
    :param matrix_as_int: bool
    :return: numpy.array
    """
    nodelist = list(nodes['osmid'])
    print(f"len of nodelist: {len(nodelist)}")
    if matrix_as_int:
        results = np.full((len(nodelist), len(nodelist)), -1, dtype=np.int32)
    else:
        results = np.full((len(nodelist), len(nodelist)), -1, dtype=np.float32)

    length = nx.algorithms.shortest_paths.weighted.all_pairs_dijkstra_path_length(G, weight='travel_time')
    start_time = time.time()
    for source in length:
        source_node = source[0]
        source_node_index = nodelist.index(source_node)
        for target_node_index in range(len(nodelist)):
            target_node = nodelist[target_node_index]
            if target_node in source[1].keys():
                results[source_node_index, target_node_index] = source[1][target_node]
        if (source_node_index % 500) == 0:
            cur_runtime = time.time() - start_time
            print(f"Done with {source_node_index} nodes in {cur_runtime} seconds")
    travel_time_matrix = results.astype(int)
    num_neg = np.sum(travel_time_matrix < 0)
    perc_neg = num_neg / (len(nodelist) * len(nodelist))
    print(f"There are {num_neg} negative travel times in travel_time_matrix which is {perc_neg} percent of all entries.")
    return travel_time_matrix



In [46]:
G, nodes, edges = osmnx_routing_graph(addr='Chattanooga, Tennessee', 
                        buffer_dist=500, 
                        strongly_connected=True, 
                        integer_edge_weights=True)

Number of nodes: 9850, number of edges: 25322


In [47]:
travel_time_matrix = osmnx_travel_time_matrix_from_graph(G, nodes, matrix_as_int=True)

len of nodelist: 9850
Done with 0 nodes in 0.13228297233581543 seconds
Done with 500 nodes in 43.02166295051575 seconds
Done with 1000 nodes in 89.3054358959198 seconds
Done with 1500 nodes in 129.72183203697205 seconds
Done with 2000 nodes in 169.97513890266418 seconds
Done with 2500 nodes in 211.71518278121948 seconds
Done with 3000 nodes in 252.64689993858337 seconds
Done with 3500 nodes in 291.0017879009247 seconds
Done with 4000 nodes in 328.02022409439087 seconds
Done with 4500 nodes in 365.3086190223694 seconds
Done with 5000 nodes in 402.6718719005585 seconds
Done with 5500 nodes in 439.9815459251404 seconds
Done with 6000 nodes in 480.10015177726746 seconds
Done with 6500 nodes in 576.7290308475494 seconds
Done with 7000 nodes in 687.2298979759216 seconds
Done with 7500 nodes in 743.9667088985443 seconds
Done with 8000 nodes in 790.573832988739 seconds
Done with 8500 nodes in 833.9086956977844 seconds
Done with 9000 nodes in 880.880108833313 seconds
Done with 9500 nodes in 924

In [48]:
# save travel_time_matrix, nodes and edges


file_path = os.path.join(OUTPUT_DIR, 'travel_time_matrix.csv')
np.savetxt(file_path, travel_time_matrix, fmt='%i', delimiter=",")

file_path = os.path.join(OUTPUT_DIR, 'nodes.csv')
nodes.to_csv(file_path, index=False)

file_path = os.path.join(OUTPUT_DIR, 'edges.csv')
edges.to_csv(file_path, index=False)

In [ ]:
# format and save for PNAS Simulator Format

nodes['node_id'] = nodes['node_id'].apply(lambda x: x + 1)
nodes = nodes[['node_id', 'lat', 'lon']]
file_path = os.path.join(OUTPUT_DIR, 'nodes_pnas.csv')
nodes.to_csv(file_path, header=False, index=False)

edges['source_node'] = edges['source_node'].apply(lambda x: x + 1)
edges['target_node'] = edges['target_node'].apply(lambda x: x + 1)
edges['travel_time'] = edges['travel_time'].apply(lambda x: math.ceil(x))
edges = edges[['source_node', 'target_node', 'travel_time']]
file_path = os.path.join(OUTPUT_DIR, 'edges_pnas.csv')
edges.to_csv(file_path, header=False, index=False)

# Testing

In [ ]:
counter = 0
val = None
loc = None
for r in range(len(travel_time_matrix)):
    for c in range(len(travel_time_matrix[0])):
        if travel_time_matrix[r, c] < 0:
            counter += 1
            val = travel_time_matrix[r, c]
            loc = [r, c]
print(counter / (len(travel_time_matrix) * len(travel_time_matrix)))
print(val)
print(loc)

In [ ]:
# test one with a path

origin = 700
destination = 900

print(travel_time_matrix[origin, destination])

origin_osmid = nodes[nodes['node_id']==origin].iloc[0]['osmid']
destination_osmid = nodes[nodes['node_id']==destination].iloc[0]['osmid']

nx.shortest_path_length(G, origin_osmid, destination_osmid, weight='travel_time')

In [ ]:
# test one without a path

origin = 10565
destination = 10134

print(travel_time_matrix[origin, destination])

origin_osmid = nodes[nodes['node_id']==origin].iloc[0]['osmid']
destination_osmid = nodes[nodes['node_id']==destination].iloc[0]['osmid']

nx.shortest_path_length(G, origin_osmid, destination_osmid, weight='travel_time')